In [1]:
import pyterrier as pt
import os
import numpy as np
import json
import pandas as pd
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [2]:
if not pt.started():
    pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


## Create Index

In [3]:
!rm -rf ./data/index/index_t1/*

In [4]:
def logeval_gen():
    path = "data/publish/English/Documents/Json/"
    for file in os.listdir(path):
        file_path = "data/publish/English/Documents/Json/" + file
        with open(file_path, "r") as f:
            docs = json.load(f)
            for doc in docs:
                text = doc["contents"].replace("\xa0", "")[:4096]
                if text:
                    yield {
                        "docno": doc["id"],
                        "text": text
                    }
                else:
                    continue

        

In [6]:
iter_indexer = pt.IterDictIndexer("./data/index/index_t1", 
    meta={'docno': 20, 'text': 4096}, 
    meta_tags={'text' : 'ELSE'}, 
    blocks=True, verbose=True, threads=12)

indexref1 = iter_indexer.index(logeval_gen())

17:25:12.487 [ForkJoinPool-2-worker-31] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (doc062208901100) - further warnings are suppressed
17:25:12.503 [ForkJoinPool-2-worker-3] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (doc062208901491) - further warnings are suppressed
17:25:12.654 [ForkJoinPool-2-worker-19] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (doc062208904175) - further warnings are suppressed
17:25:12.670 [ForkJoinPool-2-worker-5] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (doc062208904479) - further warnings are suppressed
17:25:13.947 [ForkJoinPool-2-worker-23] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (doc062201905981) - further warnings are suppressed
17:25:15.102 [ForkJoinPool-2-worker-27] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to th

In [14]:
files = pt.io.find_files("data/publish/English/Documents/Trec/")
gen = pt.index.treccollection2textgen(files, meta_tags={'text' : 'ELSE'})
indexer = pt.IterDictIndexer("./data/index/index_t1", 
    #meta= {'docno' : 26, 'text' : 10000}, 
    meta_tags={'text' : 'ELSE'}, blocks=True, verbose=True)

indexref_t1 = indexer.index(gen)

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/juerikeller/miniconda3/envs/test/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/juerikeller/miniconda3/envs/test/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/home/juerikeller/miniconda3/envs/test/lib/python3.8/site-packages/pyterrier/index.py", line 991, in _write_fifos
    for doc in it:
  File "/home/juerikeller/miniconda3/envs/test/lib/python3.8/site-packages/pyterrier/index.py", line 830, in <genexpr>
    return ({f: doc[f] for f in all_fields} for doc in it)
  File "/home/juerikeller/miniconda3/envs/test/lib/python3.8/site-packages/pyterrier/index.py", line 188, in treccollection2textgen
    rtr[k] = d.getProperty(k)
  File "jnius/jnius_export_class.pxi", line 885, in jnius.JavaMethod.__call__
  File "jnius/jnius_export_class.pxi", line 967, in jnius.JavaMethod.call_method
  File "jnius/jnius_conversion.pxi"

In [6]:
for i in gen:
    break

In [4]:
# Create Index
# !rm -rf ./data/index/index_t1/*
indexer = pt.TRECCollectionIndexer("./data/index/index_t1", meta= {'docno' : 26, 'text' : 100000}, meta_tags={'text' : 'ELSE'}, blocks=True, verbose=True)

doc_paths_t1 = [os.path.join("data/publish/English/Documents/Trec/", path) for path in os.listdir("data/publish/English/Documents/Trec/")]
indexref_t1 = indexer.index(doc_paths_t1)

  4%|▍         | 7/158 [00:28<12:15,  4.87s/files]

17:33:29.408 [main] WARN org.terrier.structures.indexing.Indexer - Adding an empty document to the index (doc062209507864) - further warnings are suppressed


 35%|███▍      | 55/158 [05:34<10:27,  6.09s/files]

In [6]:
# doc_paths_t1 = [os.path.join("data/publish/English/Documents/Trec/", path) for path in os.listdir("data/publish/English/Documents/Trec/")]
# indexref_t1 = indexer.index(doc_paths_t1)

In [4]:
index_t1 = pt.IndexFactory.of("./data/index/index_t1")

query_path_t1 = "data/publish/English/Queries/train.trec"
topics_t1 = pt.io.read_topics(query_path_t1)

qrels_t1 = pt.io.read_qrels("data/publish/French/Qrels/train.txt")


In [6]:
print(index_t1.getMetaIndex().getKeys())


['docno']


In [4]:

train_topics, validation_topics, test_topics = np.split(
            topics_t1, [int(0.6 * len(topics_t1)), int(0.8 * len(topics_t1))]
        )
train_qrels, validation_qrels, test_qrels = np.split(
            qrels_t1, [int(0.6 * len(qrels_t1)), int(0.8 * len(qrels_t1))]
        )


print(index_t1.getCollectionStatistics().toString())

Number of documents: 1570734
Number of terms: 3694081
Number of postings: 433928454
Number of fields: 0
Number of tokens: 777790536
Field names: []
Positions:   true



In [5]:
pd.__version__

'2.0.0'

In [10]:
pd_indexer = pt.DFIndexer("./tmp", type=pt.IndexingType.MEMORY)

In [33]:
queries = pt.io.read_topics(query_path_t1)
queries = queries.reset_index().rename(columns={"index": "docno"})
queries["docno"] = queries["docno"].astype(str)  # docno must be a string?!

In [23]:
queries.to_dict("records")

[{'docno': '0', 'qid': 'q06223196', 'query': 'car shelter'},
 {'docno': '1', 'qid': 'q062228', 'query': 'airport'},
 {'docno': '2', 'qid': 'q062287', 'query': 'antivirus comparison'},
 {'docno': '3', 'qid': 'q06223261', 'query': 'free antivirus'},
 {'docno': '4', 'qid': 'q062291', 'query': 'orange antivirus'},
 {'docno': '5', 'qid': 'q062295', 'query': 'antivirus smartphone'},
 {'docno': '6', 'qid': 'q0622107', 'query': 'watercolour and paintbrushes'},
 {'docno': '7', 'qid': 'q0622117', 'query': 'arena bordeaux'},
 {'docno': '8', 'qid': 'q0622118', 'query': 'colloidal silver'},
 {'docno': '9', 'qid': 'q0622141', 'query': 'unlicensed car insurance'},
 {'docno': '10', 'qid': 'q06229532', 'query': 'autonomy electric car'},
 {'docno': '11', 'qid': 'q062216106', 'query': 'antivirus avast'},
 {'docno': '12', 'qid': 'q0622226', 'query': 'beaugrenelle'},
 {'docno': '13', 'qid': 'q0622232', 'query': 'beaune'},
 {'docno': '14', 'qid': 'q0622311', 'query': 'public employment exchange'},
 {'docno'

In [31]:
iter_indexer = pt.IterDictIndexer("./tmp", type=pt.IndexingType.MEMORY, meta={'docno': 20, 'text': 4096})

queries = pt.io.read_topics(query_path_t1)
queries = queries.reset_index().rename(columns={"index": "docno"})
queries = queries.rename(columns={"query": "text"})

queries["docno"] = queries["docno"].astype(str)  # docno must be a string?!


In [32]:
indexref2 = iter_indexer.index(queries.to_dict("records"))

## Features

In [16]:
from src.LETOR import LETOR

In [17]:
letor = LETOR(index_t1, query_path_t1)

def _features(row):
    docid = row["docid"]
    
    queryid = row["qid"]
    features = row["features"]  # get the features 

    letor_features = letor.get_features_letor(queryid, docid)

    return np.append(features, letor_features)

AttributeError: 'NoneType' object has no attribute 'getDirectIndex'

## Retrieval

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import svm
import xgboost as xgb

In [9]:
TF_IDF = pt.BatchRetrieve(index_t1, wmodel="TF_IDF")
BM25 = pt.BatchRetrieve(index_t1, wmodel="BM25")
PL2 = pt.BatchRetrieve(index_t1, wmodel="PL2")

In [10]:
fbr = pt.FeaturesBatchRetrieve(index_t1, 
                             controls = {"wmodel": "BM25"}, 
                             features=[
                                      "WMODEL:Tf",
                                      "WMODEL:TF_IDF", 
                                      "WMODEL:BM25", 
                                      ]
                             ) >> pt.apply.doc_features(_features)

In [34]:
fbr.transform(validation_topics.head(2))

KeyboardInterrupt: 

In [29]:
pt.Experiment(
    [fbr],
    train_topics,
    train_qrels,
    eval_metrics=["map", "recip_rank"],
    verbose=True,
)

pt.Experiment:   0%|          | 0/1 [01:59<?, ?system/s]


KeyboardInterrupt: 

In [20]:
train_topics.shape

(403, 2)

In [22]:
train_topics

,qid,query
0,q06223196,car shelter
1,q062228,airport
2,q062287,antivirus comparison
3,q06223261,free antivirus
4,q062291,orange antivirus
...,...,...
398,q062213211,consumption electric car
399,q062213307,leg leg leg leg leg leg leg leg leg leg leg le...
400,q062213401,download video
401,q062213479,whitewater


In [27]:
# Create the regressor object.
# rf = RandomForestRegressor(n_estimators=10, max_depth=2, n_jobs=12, random_state=42, verbose=3, max_samples=100)
# print("Random Forest created")
# rf_pipe = fbr >> pt.ltr.apply_learned_model(rf)
# print("Random Forest pipeline created")
# rf_pipe.fit(train_topics, train_qrels)
# print("Random Forest done")


# # Logistic regression
# lr = LogisticRegression(random_state=42, verbose=3)
# lr_pipe = fbr >> pt.ltr.apply_learned_model(lr)
# lr_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)
# print("Logistic Regression done")


# # Support Vector regression
# svr = svm.SVR(random_state=42, verbose=3)
# svr_pipe = fbr >> pt.ltr.apply_learned_model(svr)
# svr_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)
# print("Support Vector Regression done")

# LambdaMART
lmart_x = xgb.sklearn.XGBRanker(objective='rank:ndcg',
    learning_rate=0.1,
    gamma=1.0,
    min_child_weight=0.1,
    max_depth=6,
    verbose=2,
    random_state=42)
print("LambdaMART created")

lmart_xgb_pipe = fbr >> pt.ltr.apply_learned_model(lmart_x, form="ltr")
print("LambdaMART pipeline created")
lmart_xgb_pipe.fit(train_topics, train_qrels, validation_topics, validation_qrels)
print("LambdaMART done")

LambdaMART created
LambdaMART pipeline created


TypeError: fit() got an unexpected keyword argument 'verbose'

In [ ]:
BM25

# Experiment

In [ ]:
systems = [TF_IDF, BM25, PL2, rf_pipe, lr_pipe, svr_pipe, lmart_xgb_pipe]
names  = ["TF-IDF", "BM25", "PL2", "Random Forest", "Logistic Regression", "Support Vector Regression", "LambdaMART"]

results = pt.Experiment(
    systems,
    test_topics,
    test_qrels,
    eval_metrics=["map", "ndcg", "P_20", "ndcg_cut_20"],
    baseline=0,
    names=names,
    correction='bonferroni',
    verbose=True)

: 